In [312]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Exemplo - Controle de um guindaste

Neste primeiro exemplo, o objetivo é transladar uma carga utilizando um guindaste desde um navio até a seção de armazenamento. Para isso, foi usado um sistema fuzzy para quantificar a potencia necessária.

Temos as seguintes variáveis:

* Variáveis de entrada: *ângulo* e *distância*.
* Variável de saída: *potência* (desenvolvida pelo motor).

## Inputs

Temos dois inputs de dados para este exemplo, que são:

##### Distância

Equivale a distância entre o navio e a seção de armazenamento. Esta distância é medida com valores entre 0 até 10. Para o conjunto fuzzy iremos usar três variáveis, com os seguintes valores:

* **Perto** -> Entre 0 até 5
* **Médio** -> Entre 2.5 até 7.5
* **Longe** -> Entre 5 até 10

Com isso, temos definido o conjunto fuzzy para a variável distância.

##### Ângulo

Equivale ao ângulo do guindaste em relação ao eixo do motor. Este ângulo é medido com valores entre -45 graus até 45 graus. Para o conjunto fuzzy iremos usar três variáveis, com os seguintes valores:

* **Negativo** -> Entre -45 até 0
* **Zero** -> Entre -20 até 20
* **Positivo** -> Entre 0 até 45

Com isso, temos definido o conjunto fuzzy para a variável ângulo.

## Output

Para o output desse sistema, temos uma variável linguística:

##### Potência

Equivale a potência desenvolvida pelo motor para movimentar o guindaste. Esta potência é medida em forma de porcentagem. Para este conjunto fuzzy iremos usar três variáveis, com os seguintes valores:

* **Baixa** -> Entre 0% até 7%
* **Média** -> Entre 8% até 16%
* **Alta** -> Entre 16% até 25%

Com isso, temos definido o conjunto fuzzy de output para a variável potência.

Portanto, levando em consideração estas informações, a seguir serão criadas as variáveis correspondentes a cada conjunto fuzzy com seus respectivos valores.

In [313]:
angle = ctrl.Antecedent(np.arange(-45, 46, 0.1), 'angle')
distance = ctrl.Antecedent(np.arange(0, 11, 0.1), 'distance')
power = ctrl.Consequent(np.arange(0, 0.25, 0.01), 'power')

### Definindo as funções de pertinência

As funções de pertinência são fundamentais em sistemas fuzzy, pois são elas que estabelecem a relação entre as variáveis de entrada e a variável de saída. Existem diferentes tipos de funções de pertinência, como a triangular, a trapezoidal, a gaussiana, entre outras. Cada uma dessas funções possui suas próprias características e vantagens.

Em nosso exemplo é usado a função de pertinência trapezoidal. Uma das suas principais vantagens é flexibilidade, pois permite que mais de um valor seja considerado como máximo para a variável em questão. Além disso, ela possibilita melhor incerteza quanto ao valor máximo de uma variável, o que é importante em sistemas que lidam com dados imprecisos ou incertos.

In [314]:
angle['negative'] = fuzz.trapmf(angle.universe, [-45, -45, -20, 0])
angle['zero'] = fuzz.trapmf(angle.universe, [-20, -8, 8, 20])
angle['positive'] = fuzz.trapmf(angle.universe, [0, 20, 45, 45])

distance['near'] = fuzz.trapmf(distance.universe, [0, 0, 2.5, 5])
distance['medium'] = fuzz.trapmf(distance.universe, [2.5, 3.5, 6.5, 7.5])
distance['away'] = fuzz.trapmf(distance.universe, [5, 7.5, 10, 10])

power['low'] = fuzz.trapmf(power.universe, [0, 0, 0.05, 0.07])
power['medium'] = fuzz.trapmf(power.universe, [0.08, 0.11, 0.13, 0.16])
power['high'] = fuzz.trapmf(power.universe, [0.16, 0.20, 0.25, 0.25])

### Definindo as regras e criando o sistema fuzzy

As regras em um sistema fuzzy são responsáveis por definir a lógica que será utilizada para determinar a saída do sistema com base nas entradas fuzzy. Essas regras são formadas por um conjunto de proposições lógicas que relacionam as variáveis de entrada com a variável de saída, sendo do tipo **se-então**.

No exemplo dado em aula, foram criadas três regras:

1. **Se** distância = longe **ou** ângulo = negativo **Então** potência = baixa

2. **Se** distância = média **Então** potencia = baixa

3. **Se** distância = perto **ou** ângulo = positivo **Então** potência = alto

**OBS**: As regras 1 e 3 foram mudadas para ficar de acordo com o exemplo da aula, onde houve um erro no slide.

In [315]:
rule_one = ctrl.Rule(distance['away'] | angle['negative'], power['low'])
rule_two = ctrl.Rule(distance['medium'], power['medium'])
rule_three = ctrl.Rule(distance['near'] | angle['positive'], power['high'])

fuzzy_system = ctrl.ControlSystem([rule_one, rule_two, rule_three])
control = ctrl.ControlSystemSimulation(fuzzy_system)

### Definindo novas entradas para testar o controlador

A seguir é feito o teste do sistema fuzzy, com o exemplo indicado na aula. Este exemplo de teste consiste em atribuir valores para o ângulo e distância e então receber como resultado a potencia. Foi usado método de defuzzyficação do centroide, o mesmo simulado em aula. 

Os valores são:

* ângulo: 30 graus
* distância: 3

In [316]:
control.input['angle'] = 30
control.input['distance'] = 3

control.compute()

control.output['power'] * 100

17.765765765765764

## Discutindo resultado

Para o exemplo anterior, o resultado obtido foi de 17.7%, o que difere em apenas 1% do resultado mostrado na aula. É importante destacar que a precisão dos resultados depende da definição das funções de pertinência e das regras utilizadas no sistema.

No exemplo, o valor de 30 graus para o ângulo está dentro da faixa de ângulos considerados positivos, 10 graus acima do limite para ângulos médios. Além disso, a distância de 3 está no pico da função de pertinência para distância próxima e no início da função para a funcão de pertinência de distância média. Com isso, podemos concluir que a regra mais ativada é:

**Se** distância = perto **ou** ângulo = positivo **Então** potência = alto

O valor final de 17.7% corresponde ao início da faixa de potência considerada alta, o que é coerente com a regra ativada e com os valores de entrada fornecidos.

Portanto, podemos concluir que o resultado obtido é consistente e lógico para o sistema fuzzy criado.

# Exemplo customizado

Vamos agora testar um exemplo utilizando o sistema fuzzy criado.

Esse exemplo é hipotéticos e foi desenvolvido com base em valores extrapolados, de acordo com as regras já mencionadas, para facilitar a compreensão dos resultados. 

Com base nas regras definidas anteriormente, podemos inferir que uma distância com valor 9 é considerada muito longe. Para a variável linguística ângulo foi inferido que o valor de -40 graus é considerado bastante negativo. Portanto iremos usar estes dois valores de inputs.

In [317]:
control.input['angle'] = -40
control.input['distance'] = 9

control.compute()

control.output['power'] * 100

3.0277777777777786

## Discutindo resultado

Neste exemplo tivemos um resultado obtido de 3% de potência. Isso corresponde a potencia baixa. Nossos inputs, comentados anteriormente, claramente ativam a seguinte regra:

**Se** distância = longe **ou** ângulo = negativo **Então** potência = baixa

Isto pois nossa distância com valor 9 é considerada longe e o ângulo no valor -40 graus é negativo, satisfazendo assim esta regra. Nosso resultado corresponde a uma potencia baixa o que é coerente com a regra ativada e com os valores de entrada fornecidos.

Portanto, podemos concluir que o resultado obtido é consistente e lógico para o sistema fuzzy criado.